In [8]:
from pathlib import Path
# !pip install iesopt > /dev/null

In [9]:
import iesopt

In [10]:
model = iesopt.run(
    "opt/config_summerschool2025.iesopt.yaml",
    parameters=dict(
        # price_electricity_buy = "dayahead_price@data", price_electricity_sell = 10.0 # would model buying at the day-ahead price and selling (feed-in) at a fixed price of 10.0 EUR/MWh.
    ),
)

In [11]:
df_results = model.results.to_pandas()

# list components
print( df_results["component"].unique() )

# electricity buy / sell
elec_buy = df_results.query("component == 'buy_electricity' and field == 'value'")
elec_sold = df_results.query("component == 'sell_electricity' and field == 'value'")
combo = elec_buy.merge(elec_sold, how="inner", on="snapshot", suffixes=("_buy", "_sell"))
combo.to_csv("opt/out/elec_buy_sold.csv")

# pv
pv = df_results.query("component == 'pv' and field == 'value'")
# load
load = df_results.query("component == 'load' and field == 'value'") 
combo=pv.merge(load, how="inner", on="snapshot", suffixes=("_pv", "_load"))
combo.to_csv("opt/out/pv_load.csv")

# battery
batt_charge = df_results.query(
    "component.str.endswith('.charging') and fieldtype == 'exp' and field =='in_electricity'",
    engine="python"
)
batt_discharge = df_results.query(
    "component.str.endswith('.discharging') and fieldtype == 'exp' and field =='in_electricity'",
    engine="python"
)
combo =batt_charge.merge(batt_discharge, how="inner", on="snapshot", suffixes=("_charge", "_discharge"))
combo.to_csv("opt/out/batt_charge_discharge.csv")


['battery.discharging' 'battery.storage' 'pv' 'battery' 'buy_electricity'
 'load' 'sell_electricity' 'battery.charging' 'grid_electricity']


# Vizualization

In [12]:
from iesopttools import RDB, Figure, Trace
rdb = RDB()
entry = rdb.add_entry(model)

fig = Figure(style="seaborn", barmode="relative", labels=dict(title="electricity: supply / demand", x="time", y="MW"))

for asset in entry.query("carrier", "node = 'grid_electricity' AND direction = 'out'"):
    fig.add(Trace("bar", entry.select(asset)))

for asset in entry.query("carrier", "node = 'grid_electricity' AND direction = 'in'"):
    fig.add(Trace("bar", entry.select(asset), sign=-1.0))


In [15]:
display(fig._fig)

None